<img src="https://electronaut.info/wp-content/uploads/2017/05/Tubescape_jupyter2.jpg">

<h1><font color = b30000>TUBE DATA MATCHER for uTRACER FILES</font></h1><h5>Anode current for a sweep of grid voltages  when anode voltage = constant</h5>


# STEP 1 — Import and process tube data, build dataframe.

In [1]:
from electronaut_uTracer_Import import *
df.head()

Please enter the path to the folder containing the uTracer files: /Users/rrmc/github_rrMacKinnon/Electrona_uTracer/SampleTubeData_1_to_196

What type of tubes are these? 5749

— — — — — 

Data from 167 tubes of type 5749 was successfully read.
A dataframe containing all 167 tubes was successfully created.


,tube_ID,tube_type,anode_voltage,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
0,1,5749,325,99.278,65.383,44.717,34.082,26.669,20.931,16.492,12.992,10.291,8.23,6.669,5.495,4.626
1,10,5749,325,84.662,55.752,37.5,27.666,20.873,15.84,12.088,9.304,7.261,5.807,4.747,3.945,3.344
2,100,5749,325,76.506,50.319,33.695,25.027,19.052,14.603,11.283,8.775,6.954,5.655,4.765,4.118,3.619
3,102,5749,325,92.106,59.906,39.752,29.517,22.626,17.543,13.745,10.894,8.787,7.228,6.08,5.204,4.575
4,103,5749,325,69.639,45.449,30.772,22.843,17.624,13.782,10.864,8.615,6.916,5.657,4.754,4.092,3.619


##### TODO:  INSERT PLOT OF ALL TUBES


# STEP 3: Show the statistics of the tube data set

In [2]:
# For some reason, .describe is requiring the .astype(float) in order to show the stats.
df_stats = df.astype(float).describe()
df_stats.loc[:, 'tube_type'] = df_stats.loc[:, 'tube_type'].astype(int)
df_stats.loc[:, 'tube_ID'] = df_stats.loc[:, 'tube_ID'].astype(int)
# df_stats.loc[:, 'tube_ID'] = df_stats.loc[:, 'tube_ID'].astype(str)
df_stats

,tube_ID,tube_type,anode_voltage,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
count,167,167,167.0,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000
mean,94,5749,325.0,85.854024,54.600138,35.936389,26.606778,20.321790,15.648988,12.173521,9.577371,7.657347,6.248994,5.208695,4.410192,3.798886
std,57,0,0.0,10.252616,5.708263,3.792364,2.876220,2.387349,2.053135,1.794487,1.576286,1.382130,1.206177,1.052825,0.929768,0.840940
min,1,5749,325.0,48.494000,30.962000,19.730000,14.837000,11.473000,8.866000,6.827000,5.260000,4.091000,3.235000,2.613000,2.153000,1.833000
25%,43,5749,325.0,80.143500,51.841500,34.041500,25.167500,19.052000,14.428000,11.082500,8.645000,6.855500,5.501500,4.584500,3.869500,3.319500
50%,94,5749,325.0,87.154000,55.362000,36.408000,26.777000,20.534000,15.759000,12.180000,9.602000,7.679000,6.162000,5.096000,4.343000,3.712000
75%,143,5749,325.0,91.902000,58.625500,38.192000,28.252000,21.554500,16.662500,13.070000,10.400000,8.427000,6.923000,5.801500,4.957500,4.302500
max,196,5749,325.0,107.892000,65.654000,46.996000,36.666000,30.080000,25.292000,21.460000,18.384000,15.898000,13.867000,12.198000,10.760000,9.616000


##### TODO:  INSERT PLOT(S) OF STATISTICS 

# STEP 4: Calculate the differences between all tubes

In [3]:
def build_difference_df(df, df_stats):
       
    # Make a list of all the columns in the dataframe
    cols = list(df.columns)
    
    # Make a list of just the bias columns
    bias_list = []
    for i in cols:
        if i.startswith('Bias_') == True:
            bias_list.append(i)
    
    # BMake an empty list to hold the calculated values, to be used to build the difference dataframe
    diff_squared_list = []
    
    # Iterate through the index of tubes in the master dataframe to select a reference tube
    for each_ref_tube in df.index:
        
        # Get the next reference tube data, as a series
        ref_tube = df.iloc[each_ref_tube]
        ref_tube_ID = ref_tube.tube_ID

        # Iterate through the index of tubes in the master dataframe to select a match tube
        for each_match_tube in df.index:
            
            # Get the next tube data to be compared, as a series
            match_tube = df.iloc[each_match_tube]
            match_tube_ID = match_tube.tube_ID
            
            # Dictionary container to hold the difference variables for each tube
            temp_dict = {}
            
            # Compute the squares of the differences in each current measurement
            error_sum = 0
            for bias in enumerate(bias_list):
                mismatch = ((float(match_tube.loc[bias[1]]) - float(ref_tube.loc[bias[1]]))**2)
                temp_dict[bias[1]] = mismatch
                error_sum = error_sum + mismatch

            # Add other relevant key/value pairs to the dictionary  
            temp_dict['ref_tube_ID'] = ref_tube_ID
            temp_dict['match_tube_ID'] = match_tube_ID
            temp_dict['error_sum'] = error_sum

            # Append the tube's calculated values to the diff_squared_list
            diff_squared_list.append(temp_dict)
    
    # Make an ordered list for the columns of the new difference dataframe
    new_cols_to_add = ['ref_tube_ID', 'match_tube_ID', 'error_sum']
    df_dif_cols = new_cols_to_add + bias_list
    df_dif = pd.DataFrame(diff_squared_list, columns = df_dif_cols)
    return df_dif
    
    
df_dif = build_difference_df(df, df_stats)
df_dif.head()

,ref_tube_ID,match_tube_ID,error_sum,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
0,1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,10,514.934156,213.627456,92.756161,52.085089,41.165056,33.593616,25.918281,19.395216,13.601344,9.180900,5.870929,3.694084,2.402500,1.643524
2,1,100,1116.246220,518.563984,226.924096,121.484484,81.993025,58.018689,40.043584,27.133681,17.783089,11.135569,6.630625,3.625216,1.896129,1.014049
3,1,102,170.397792,51.437584,29.997529,24.651225,20.839225,16.345849,11.478544,7.546009,4.401604,2.262016,1.004004,0.346921,0.084681,0.002601
4,1,103,1805.026199,878.470321,397.364356,194.463025,126.315121,81.812025,51.108201,31.674384,19.158129,11.390625,6.620329,3.667225,1.968409,1.014049


# Find the best matches for a particular tube_ID

In [7]:
def find_best_set_for_chosen_tube_ID(tube_to_match, tube_set_size):
    tube_to_match = str(tube_to_match)
    
    # Create a new dataframe showing only tube_set_size number of nearest matches to the chosen tube_to_match
    df_tube_set = df_dif[(df_dif.ref_tube_ID == tube_to_match)].sort_values('error_sum').head(tube_set_size)
    
    return df_tube_set

tube_to_match = 33
tube_set_size = 16
df_tube_set = find_best_set_for_chosen_tube_ID(tube_to_match, tube_set_size)
df_tube_set.head(tube_set_size)


,ref_tube_ID,match_tube_ID,error_sum,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
17472,33,33,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
17412,33,146,1.700799,0.034969,0.436921,0.293764,0.182329,0.012544,0.013225,0.081225,0.135424,0.153664,0.130321,0.097344,0.072900,0.056169
17415,33,149,2.294372,1.795600,0.065025,0.016641,0.128881,0.081225,0.034969,0.001024,0.005329,0.017161,0.024336,0.027556,0.043264,0.053361
17373,33,104,4.856195,2.178576,0.023409,0.077284,0.038809,0.114921,0.188356,0.310249,0.384400,0.388129,0.351649,0.285156,0.266256,0.249001
17477,33,38,5.082372,2.762244,0.746496,0.450241,0.339889,0.165649,0.114921,0.074529,0.063001,0.063504,0.066564,0.073441,0.082369,0.079524
17501,33,64,5.103931,4.713241,0.331776,0.000169,0.000169,0.000036,0.000289,0.000004,0.000289,0.003844,0.011664,0.019600,0.015625,0.007225
17456,33,196,5.279906,4.280761,0.414736,0.050176,0.050176,0.047089,0.019321,0.000529,0.004356,0.024336,0.052441,0.084100,0.114244,0.137641
17377,33,109,5.703943,2.762244,0.103684,0.047089,0.093025,0.014884,0.007744,0.108241,0.265225,0.385641,0.452929,0.495616,0.497025,0.470596
17394,33,128,7.187544,0.013924,2.486929,1.313316,0.423801,0.249001,0.245025,0.321489,0.372100,0.400689,0.380689,0.334084,0.314721,0.331776
17529,33,94,7.350801,4.787344,1.329409,0.586756,0.348100,0.194481,0.079524,0.017424,0.001681,0.000100,0.000144,0.000625,0.001369,0.003844


In [8]:
tube_set_score = df_tube_set.error_sum.sum()
tube_set_score

117.4344890000002

# STEP 5: Find the best matches for any tube set size
# Build a dataframe of the best matched sets in order

In [27]:
# Make a local version of df_dif that can be butchered
df_dif_copy = df_dif.copy()

def ask_tube_set_size():
    
    # Ask the user to enter the set size
    tube_set_size = input("Please enter the number of tubes per matched set:")
    
    # Build a list of tube names for the tube set size
    tube_sets_columns = ['Tube_' + str(1 + i) for i in range(int(tube_set_size))]    
    
    return tube_set_size, tube_sets_columns 

def find_best_set_for_chosen_tube_ID(df_dif_copy, tube_to_match, tube_set_size, tube_sets_columns):
    tube_to_match = str(tube_to_match)
        
    # Create a new dataframe showing only tube_set_size number of nearest matches to the chosen tube_to_match
    df_tube_set = df_dif_copy[(df_dif_copy.ref_tube_ID == tube_to_match)].sort_values('error_sum').head(int(tube_set_size))
        
    # Create the dictionary of the tubes in the set and the total score of the set
    tube_set_list = list(df_tube_set.match_tube_ID)
    tube_set_dict = dict(zip(tube_sets_columns, tube_set_list))   
    tube_set_score = df_tube_set.error_sum.sum()
    tube_set_dict.update({'tube_set_score':tube_set_score})

    return tube_set_dict


def find_best_set(df_dif_copy, tube_set_size, tube_sets_columns):
    
    # Make a list of all the tubes
    tube_list = list(df_dif_copy.ref_tube_ID.unique().astype(str))

    # Make an empty list to hold the tube sets and their scores
    tube_sets_and_scores_list = []

    for i in tube_list:
        temp_dict = find_best_set_for_chosen_tube_ID(df_dif_copy, i, tube_set_size, tube_sets_columns)
        tube_sets_and_scores_list.append(temp_dict)

    # Add 'tube_set_score' column name at the front of the list
    tube_sets_columns.insert(0, 'tube_set_score')    
    
    # Create a dataframe for the tube sets called df_tube_sets 
    df_tube_sets = pd.DataFrame(tube_sets_and_scores_list, columns=tube_sets_columns)
    
    # Sort the tube_set_score column, take the top row (with the lowest score) as a new df 
    df_best_set_temp = df_tube_sets.sort_values('tube_set_score').head(1)
        
    return df_best_set_temp
   

def remove_best_set_from_df_dif(df_dif_copy, df_best_set):
    
    # Make a list of all the tubes in the set, to be removed for the next match sequence
    tubes_to_be_removed = list(df_best_set.iloc[0])
    tubes_to_be_removed.pop(0)
    
    for tube in tubes_to_be_removed:
        df_dif_copy.drop(tube)

    return df_dif_copy
    

    
# Make an empty dataframe to hold all the sets and their scores
df_best_tube_sets = pd.DataFrame()

# Get the tube_set_size and the tube_sets_columns from the ask_tube_set_size function
tube_set_size, tube_sets_columns = ask_tube_set_size()

# Get the best tube set from the batch
df_best_set_temp = find_best_set(df_dif_copy, tube_set_size, tube_sets_columns) 

# Append the best tube set to the df_best_tube_sets dataframe
df_best_tube_sets = df_best_tube_sets.append(df_best_set_temp)

# Remove the best tube set from df_dif_copy
# df_dif_copy = remove_best_set_from_df_dif(df_dif_copy, df_best_set_temp)
    
    
    
    
    
# Remove function isn't working.
# Loop isn't set up
    
    
    
    
    
# if len(list(df_dif_copy.ref_tube_ID.unique())) > int(tube_set_size): 
        
    # Get all the rest of the sets from the batch
#     df_best_set_temp = find_best_set(df_dif_copy)
#     df_best_tube_sets.append(df_best_set_temp.iloc[0])

    # Make a list of all the tubes in the set, to be removed for the next match sequence
#     tubes_to_be_removed = (df_best_set.iloc[0])
#     print(type(tubes_to_be_removed))
    

          
#     tubes_to_be_removed.pop(0)
#     for tube in tubes_to_be_removed:
#           df_dif_copy.drop(tube)
        




Please enter the number of tubes per matched set:8


In [29]:
df_best_tube_sets

,tube_set_score,Tube_1,Tube_2,Tube_3,Tube_4,Tube_5,Tube_6,Tube_7,Tube_8
103,11.73209,32,17,5,123,14,15,117,194


In [28]:
df_best_set_temp


,tube_set_score,Tube_1,Tube_2,Tube_3,Tube_4,Tube_5,Tube_6,Tube_7,Tube_8
103,11.73209,32,17,5,123,14,15,117,194


In [ ]:


df_dif_copy2 = df_dif_copy.copy()
# for tube in tubes_to_be_removed:
#     print(tube)

# thing = df_dif_copy2[(df_dif_copy2.ref_tube_ID != df_best_set.loc[0, 'Tube_1'])]

# Miscellaneous exeriments with ipywidgets

In [ ]:
import ipywidgets as wg
from ipywidgets import Layout, Box
from IPython.display import display


# Make a list of all the tube_ID names, then build a select box with those values
tube_list = list(df.loc[:, 'tube_ID'])
tube_list.sort(key=int)
tube_ID_select_box = wg.Select(options=tube_list, value=tube_list[0], description='Reference Tube', layout=Layout(width='150px'), disabled=False)

# Make a list of the tubes for the "Matched Tubes" selectMultiple box
matched_list = ['1', '33', '67', '22', '45']
matched_tube_box = wg.SelectMultiple(options=matched_list, layout=Layout(display="flex", flex_flow='column', width='350px'), description="Matched Tubes") 


items = [tube_ID_select_box, matched_tube_box]

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
box = Box(children=items, layout=box_layout)

box


# def respond(sender):
    # pass the chosen Tube_ID to a function that looks up the resulting match list, then fill the match list box
    
    
# display(tube_ID_select_box,matched_tube_box)
# tube_ID_select_box.observe(respond)

# Other Miscellaneous stuff

##### Apply a function to a column
df.Bias_2.max()

In [ ]:
# Applymap is useful for applying a function to all data
# For example, converting all data from floats to ints
# df.loc[2:7, 'Bias_2': 'Bias_50'].applymap(int)

In [ ]:
# List all available magics
% lsmagic

In [ ]:
# Display environmental variables
env_variables = %env
current_path = env_variables.get('PATH')
current_path

In [ ]:
mydict = % env
mydict.keys()

In [ ]:
% env

In [ ]:
# % time x = range(10000)
% timeit x = range(10000)
max(x)